# Benchmark Python

* Jak mierzyć czas wykonywania kodu w Pythonie?
* Jak porównywać algorytmy w Pythonie?

Eksperymenty wykonamy dla algorytmu sprawdzającego, czy zadana liczba jest liczbą pierwszą.

## Testy pierwszości

### Algorytm 1: `is_prime_v1`

Algorytm naiwny (_brute force_) sprawdzający, czy liczba `n` jest liczbą pierwszą. Jeśli `n` jest mniejsze od 2, zwracamy `False`. W przeciwnym razie, sprawdzamy, czy `n` jest podzielne przez jakąkolwiek liczbę od 2 do `n-1`. Jeśli tak, zwracamy `False`. W przeciwnym razie, zwracamy `True`.

In [ ]:
def is_prime_v1(n) -> bool:
    if n < 2: return False
    for i in range(2, n):
        if n % i == 0: return False
    return True

assert is_prime_v1(53)==True
assert is_prime_v1(55)==False

### Algorytm 2: `is_prime_v2`

Wprowadzamy optymalizację poprzedniego algorytmu. Zauważamy, że jeśli `n` jest podzielne przez jakąkolwiek liczbę od 2 do `n-1`, to jest również podzielne przez jakąkolwiek liczbę od 2 do `sqrt(n)`. Dlatego możemy zatrzymać pętlę, gdy `i` osiągnie `sqrt(n)`.

In [ ]:
def is_prime_v2(n):
    if n < 2: return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0: return False
    return True

assert is_prime_v1(53)==True
assert is_prime_v1(55)==False

### Algorytm 3: `is_prime_v3`

Kolejna optymalizacja polega na zauważeniu, że liczby parzyste nie są liczbami pierwszymi (poza 2). Dlatego możemy zacząć od 3 i zwiększać `i` o 2, aby sprawdzić, czy `n` jest podzielne przez jakąkolwiek liczbę nieparzystą od 3 do `sqrt(n)`.

In [ ]:
def is_prime_v3(n):
    if n < 2 : return False
    if n == 2: return True
    i = 3
    while i ** 2 <= n:
        if n % i == 0 : return False
        i += 2
    return True

assert is_prime_v1(53)==True
assert is_prime_v1(55)==False

### Algorytm 4: `is_prime_v4`

Kolejna optymalizacja polega na skorzystaniu z faktu, że liczby pierwsze większe od 3 mają postać `6k ± 1`. Dzięki temu możemy zacząć od 5 i zwiększać `i` o 6, aby sprawdzić, czy `n` jest podzielne przez jakąkolwiek liczbę postaci `6k ± 1` od 5 do `sqrt(n)`.

In [ ]:
def is_prime_v4(n):
    if n < 2: return False
    if n in (2, 3): return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i ** 2 <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

assert is_prime_v1(53)==True
assert is_prime_v1(55)==False

### Referencje dotyczące testu pierwszości

* https://en.wikipedia.org/wiki/Primality_test
* [Big primes](https://bigprimes.org/)


## Sposób 1: `time.time()`

Spotykanym czasami sposobem na mierzenie czasu wykonywania kodu w Pythonie jest użycie funkcji `time.time()` z modułu `time`. Funkcja ta zwraca liczbę sekund od 1 stycznia 1970 roku. Możemy ją wywołać przed i po wykonaniu kodu, a następnie obliczyć różnicę między tymi dwoma czasami. Niestety, ten sposób nie jest zbyt dokładny, ponieważ funkcja `time()` korzysta z systemowej daty i czasu, która może być zmieniana przez użytkownika lub inne procesy (na przykład synchronizacja zegara systemowego z czasem uniwersalnym wykonywana podczas pomiaru czasu działania algorytmu). Ponadto, dokładność tej funkcji zależy od systemu operacyjnego, na którym jest uruchamiana. W związku z tym, nie jest to najlepszy sposób na obiektywne mierzenie czasu wykonywania kodu w Pythonie.

In [ ]:
import time

t = time.time()
print(is_prime_v1(49560499))
print(time.time() - t)

## Sposób 2: `time.perf_counter()`

Moduł `time` zawiera również funkcję `time.perf_counter()`, która zwraca czas w sekundach z dokładnością do nanosekund. Funkcja ta jest zalecana do mierzenia czasu wykonywania kodu w Pythonie, ponieważ jest niezależna od systemu operacyjnego i zwraca czas z większą dokładnością (na podstawie taktów maszyny). Możemy ją wywołać przed i po wykonaniu kodu, a następnie obliczyć różnicę między tymi dwoma czasami. Funkcja `perf_counter()` została wprowadzona do Pythona właśnie po to, aby zastąpić funkcję `time()` do pomiaru czasu wykonywania kodu. Jest to zalecany sposób na obiektywne mierzenie czasu wykonywania kodu w Pythonie.

In [ ]:
import time

t = time.perf_counter()
print(is_prime_v1(49560499))
print(time.perf_counter() - t)

Drobne usprawnienie dla prcesu testowania - dekorator `@benchmark`

```python
import time

# deklaracja dekoratora
def benchmark(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f"{func.__name__} executed in {end - start:.6f} seconds")
        return result
    return wrapper
```

Teraz możemy oznaczyć funkcje, które chcemy przetestować, dekoratorem `@benchmark`:

```python
@benchmark
def is_prime_v1(n):
    if n < 2:
        return False
    for i in range(2, n):
        if n % i == 0:
            return False
    return True
```

In [ ]:
# deklaracja dekoratora
def benchmark(func):
    import time
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f"{func.__name__}: czas wykonania {end - start:.6f} sekund")
        return result
    return wrapper

@benchmark
def is_prime_v1(n):
    if n < 2:
        return False
    for i in range(2, n):
        if n % i == 0:
            return False
    return True

print(is_prime_v1(49560499))

## Sposób 3: moduł `timeit`

Moduł `timeit` dostarcza proste metody do pomiaru czasu wykonywania kodu w Pythonie. Możemy użyć funkcji `timeit.timeit()` do pomiaru czasu wykonywania kodu. Funkcja ta przyjmuje trzy argumenty: kod, który chcemy zmierzyć, liczbę powtórzeń i liczbę powtórzeń dla każdego pomiaru. Funkcja `timeit.timeit()` zwraca czas wykonywania kodu w sekundach. Możemy również użyć funkcji `timeit.repeat()` do pomiaru czasu wykonywania kodu wielokrotnie. Funkcja ta zwraca listę czasów wykonywania kodu w sekundach. Możemy użyć tej funkcji do porównania czasu wykonywania różnych wersji kodu.

https://docs.python.org/3/library/timeit.html


Przykład użycia funkcji `timeit.timeit()` do pomiaru czasu wykonywania kodu - jednej funkcji:

In [ ]:
import timeit

print(timeit.timeit('is_prime_v1(49560499)', 
                    globals=globals(), 
                    number=1))

Przykład użycia funkcji `timeit.timeit()` do pomiaru czasu wykonywania kodu - wielu funkcji:

In [ ]:
import timeit

for f in (is_prime_v1, is_prime_v2, is_prime_v3, is_prime_v4):
    print(f"{f.__name__}: ", timeit.timeit('f(49560499)', globals=globals(), number=1))


## Sposób 4: polecenie `time` w linii poleceń (Linux)

Całościowy czas działania aplikacji można zmierzyć za pomocą polecenia `time` w linii poleceń systemu operacyjnego Linux. Polecenie to mierzy czas wykonywania programu, w tym czas CPU, czas rzeczywisty i czas systemowy. Możemy użyć polecenia `time` do pomiaru czasu wykonywania skryptu Pythona. Polecenie to zwraca czas CPU, czas rzeczywisty i czas systemowy w sekundach. Możemy użyć tej metody do porównania czasu wykonywania różnych wersji kodu.

```bash
time python3 script.py
```

## Sposób 5: `pytest-benchmark`

https://superfastpython.com/python-pytest-benchmark/

## Sposób 6: `benchmarkit`

https://superfastpython.com/python-benchmarkit/